References:
- https://docs.python.org/3/reference/datamodel.html
- Fluent Python by Luciano Ramalho. Chapter 16: Coroutines

# Coroutines for Discrete Event Simulation


In [2]:
import sys
import random
import collections
import queue

In [3]:
Event = collections.namedtuple('Event', 'time proc action')

In [4]:
def compute_delay(interval):
    """Compute action delay using exponential distribution"""
    return int(random.expovariate(1/interval)) + 1

<1> `taxi_process` will be called once per taxi, creating a generatorobject to represent its operations. `ident` is the index number of the taxi (eg. 0, 1, 2 in the sample run); `trips` is the number of trips this taxi will make before going home; `start_time` is when the taxi leaves the garage.


<2> The first `Event` yielded is `'leave garage'`. This suspends the coroutine, and lets the simulation main loop proceed to the next scheduled event. When it's time to reactivate this process, the main loop will `send` the current simulation time, which is assigned to `time`.


<5> An `Event` signaling passenger pick up is yielded. The coroutine pauses here. When the time comes to reactivate this coroutine, the main loop will again `send` the current time.


<7> An `Event` signaling passenger drop off is yielded. Coroutine suspended again, waiting for the main loop to send the time of when it's time to continue.


<8> The `for` loop ends after the given number of trips, and a final `'going home'` event is yielded, to happen 1 minute after the current time. The coroutine will suspend for the last time. When reactivated, it will be sent the time from the simulation main loop, but here I don't assign it to any variable because it will not be useful.

In [5]:
def taxi_process(ident, trips, start_time=0):  # <1>
    """Yield to simulator issuing event at each state change"""
    time = yield Event(start_time, ident, 'leave garage')  # <2>
    for i in range(trips):  # repeated once for each trip
        prowling_ends = time + compute_delay(SEARCH_DURATION)  # The endiing time of the search  for a passenger.
        time = yield Event(prowling_ends, ident, 'pick up passenger')  # <5>

        trip_ends = time + compute_delay(TRIP_DURATION)  # ending time of trip
        time = yield Event(trip_ends, ident, 'drop off passenger')  # <7>

    yield Event(time + 1, ident, 'going home')  # <8>
    # end of taxi process # raises `StopIeteration`

In [6]:
class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)


    def run(self, end_time):
        """Schedule and display events until time is up"""
        # schedule the first event for each cab
        for _, proc in sorted(self.procs.items()):  # use sorted to retrieve the self.procs items ordered by integer key
            first_event = next(proc)  # 
            self.events.put(first_event)  # <4>

        # main loop of the simulation
        time = 0
        while time < end_time:  # <5>
            if self.events.empty():  # <6>
                print('*** end of events ***')
                break

            # get and display current event
            current_event = self.events.get()  # <7>
            print('taxi:', current_event.proc,  # <8>
                  current_event.proc * '   ', current_event)

            # schedule next action for current proc
            time = current_event.time  # <9>
            proc = self.procs[current_event.proc]  # <10>
            try:
                next_event = proc.send(time)  # <11>
            except StopIteration:
                del self.procs[current_event.proc]  # <12>
            else:
                self.events.put(next_event)  # <13>
        else:  # <14>
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))